In [17]:

from llm_evals.checks import ContainsCheck, PythonCodeBlocksPresent
from llm_evals.eval import Eval, PromptTest


eval_obj = Eval(
    test_sequence=PromptTest(
        prompt='Create a python function called `mask_emails` that masks emails',
        checks=[
            ContainsCheck(value='def mask_emails('),
            PythonCodeBlocksPresent(),
        ],
    ),
)
print(eval_obj)

Eval(
    uuid=None,
    test_sequence=[
        PromptTest(
            prompt='Create a python function called `mask_emails` that masks emails',
            checks=[
                ContainsCheck(value='def mask_emails(', metadata={}),
                PythonCodeBlocksPresent(min_code_blocks=1, metadata={})
            ],
        )
    ],
)


In [18]:
fake_llm = lambda _: "```python\ndef mask_emails(emails):\n...\n```"  # noqa
result = eval_obj(fake_llm)
print(result)

EvalResult:
    # of Response Characters=42
    Total Time (seconds)=5.4836273193359375e-06
    Characters per Second=6477855.3626524145
    # of Checks=2
    # of Pass/Fail Checks=2
    # of Passing Checks=1
    % of Passing Checks=0.5
